# Количество деревьев
## 1. (10 баллов) Сколько существует возможных укорененных и неукорененных топологий деревьев на N листьях?

### Укорененные деревья
Укорененное бинарное дерево на $n$ листьях может быть сформировано путем добавления $n$-го листа 
в середине любого из ребер укорененного дерева на $n - 1$ листьях.
Для дерева на $n-1$ листьях сузетсвует $2n-3$ рёбер, к которым может быть присоеденен новый лист: $2n-4$ рёбер и корень.
Значит $R(n) = (2n-3)R(n-1)$.
Зная базу индукции получаем итоговую формулу:
$$R(n) = 1 \cdot 3 \cdot 5 \cdot \ldots \cdot (2n-3) = (2n-3)!!$$

### Неукорененные деревья
Аналогично, только количетво подходящиъ рёбер равно $2n-5$ на каждом шаге:
$$U(n)=1 \cdot 3 \cdot 5 \cdot \ldots \cdot (2n-5) = (2n-5)!!$$


# Построение деревьев

## (18 баллов) Напишите программу, реализующую алгоритмы WPGMA и UPGMA.

In [1]:
from pprint import pprint
from pampy import match, _
from collections import defaultdict

import numpy as np

In [2]:
def rec_ans(n, ds):
    if isinstance(n, str): return n
    
    it = iter(n)
    chld1 = next(it)
    chld2 = next(it)
    
    return f'({rec_ans(chld1, ds)}{":" + str(ds[(node_name(n), node_name(chld1))])},' \
           f'{rec_ans(chld2, ds)}{":" + str(ds[(node_name(n), node_name(chld2))])})'

def dist_to_leaf(n, ds):
    if isinstance(n, str): return 0
    any_chld = next(iter(n))
    return ds[(node_name(n), node_name(any_chld))] + dist_to_leaf(any_chld, ds)
    
def node_name(n):
    return match(n,
                 str, lambda s: s,
                 _,   lambda n: ''.join([node_name(cn) for cn in n]))
    
def wpgma(ds):
    rec_ds = {}
    while True:
        old_node1, old_node2 = min(ds, key=ds.get)
        new_node = (old_node1, old_node2)
        
        rec_ds[(node_name(new_node), node_name(old_node1))] = min(ds.values()) / 2 - dist_to_leaf(old_node1, rec_ds)
        rec_ds[(node_name(new_node), node_name(old_node2))] = min(ds.values()) / 2 - dist_to_leaf(old_node2, rec_ds)
        
        if len(ds) <= 1: break
        
        for node in set([n for ns in ds.keys() for n in ns]) - {old_node1, old_node2}:
            ds[frozenset([new_node, node])] = \
                    (ds[frozenset([old_node1, node])] + ds[frozenset([old_node2, node])]) / 2
            
        ds = {k: v for k, v in ds.items() if (old_node1 not in k and old_node2 not in k)}
        
    return rec_ans(next(iter(ds)), rec_ds)

def upgma(ds):
    rec_ds = {}
    cluster_size = lambda n: str(n).count('(') + 1 # :)
    
    while True:
        old_node1, old_node2 = min(ds, key=ds.get)
        ons1, ons2 = cluster_size(old_node1), cluster_size(old_node2)
        
        new_node = (old_node1, old_node2)
        
        rec_ds[(node_name(new_node), node_name(old_node1))] = min(ds.values()) / 2 - dist_to_leaf(old_node1, rec_ds)
        rec_ds[(node_name(new_node), node_name(old_node2))] = min(ds.values()) / 2 - dist_to_leaf(old_node2, rec_ds)
        
        if len(ds) <= 1: break

        for node in set([n for ns in ds.keys() for n in ns]) - {old_node1, old_node2}:
            ds[frozenset([new_node, node])] = \
                (ds[frozenset([old_node1, node])] * ons1 + ds[frozenset([old_node2, node])] * ons2) / (ons1 + ons2)

        ds = {k: v for k, v in ds.items() if (old_node1 not in k and old_node2 not in k)}
    return rec_ans(next(iter(ds)), rec_ds)

# Построение деревьев

## (18 баллов) Напишите программу, реализующую алгоритмы WPGMA и UPGMA.


In [3]:
def nj(ds):
    rec_ds = defaultdict(lambda: -1)
    def m(ds, ns):
        n1, n2 = ns
        vs = [d for ns, d in ds.items() if n1 in ns and not n2 in ns]
        return np.mean(vs) if len(vs) > 0 else 0
    
    while True:
        new_ds = {ns: (d - m(ds, ns) - m(ds, ns)) for ns, d in ds.items()}
        
        old_node1, old_node2 = min(ds, key=new_ds.get)
        new_node = (old_node1, old_node2)
        
        m_ab = m(ds, (old_node1, old_node2))
        m_ba = m(ds, (old_node2, old_node1))
        
        rec_ds[(node_name(new_node), node_name(old_node1))] = (min(ds.values()) + m_ab - m_ba) / 2
        rec_ds[(node_name(new_node), node_name(old_node2))] = (min(ds.values()) - m_ab + m_ba) / 2
        
        if len(ds) <= 1: break
        
        for node in set([n for ns in ds.keys() for n in ns]) - {old_node1, old_node2}:
            ds[frozenset([new_node, node])] = \
                (ds[frozenset([old_node1, node])] + ds[frozenset([old_node2, node])] - ds[frozenset([old_node1, old_node2])]) / 2
            
        ds = {k: v for k, v in ds.items() if (old_node1 not in k and old_node2 not in k)}
    return rec_ans(next(iter(ds)), rec_ds)


## Тесты (для обеих задач):
## Тест 1

In [4]:
# frozenset for not caring about order of element (can't use simple set because it's mutable and unhashable)
t1 = {
    frozenset(['A', 'B']): 16,
    frozenset(['A', 'C']): 16,
    frozenset(['A', 'D']): 10,
    frozenset(['B', 'C']): 8,
    frozenset(['B', 'D']): 8,
    frozenset(['C', 'D']): 4
}

print('WPGMA:', wpgma(dict(t1)))
print('UPGMA:', upgma(dict(t1)))
print('Neighbor joining:', nj(dict(t1)))

WPGMA: ((B:4.0,(C:2.0,D:2.0):2.0):3.25,A:7.25)
UPGMA: ((B:4.0,(C:2.0,D:2.0):2.0):3.0,A:7.0)
Neighbor joining: ((B:5.5,(C:3.5,D:0.5):0.5):5.25,A:5.25)


## Тесты (для обеих задач):
## Тест 2

In [5]:
t2 = {
    frozenset(['A', 'B']): 5,
    frozenset(['A', 'C']): 4,
    frozenset(['A', 'D']): 7,
    frozenset(['A', 'E']): 6,
    frozenset(['A', 'F']): 8,
    frozenset(['B', 'C']): 7,
    frozenset(['B', 'D']): 10,
    frozenset(['B', 'E']): 9,
    frozenset(['B', 'F']): 11,
    frozenset(['C', 'D']): 7,
    frozenset(['C', 'E']): 6,
    frozenset(['C', 'F']): 8,
    frozenset(['D', 'E']): 5,
    frozenset(['D', 'F']): 9,
    frozenset(['E', 'F']): 8
}

print('WPGMA:', wpgma(dict(t2)))
print('UPGMA:', upgma(dict(t2)))
print('Neighbor joining:', nj(dict(t2)))

WPGMA: ((((C:2.0,A:2.0):1.0,B:3.0):1.0,(D:2.5,E:2.5):1.5):0.5,F:4.5)
UPGMA: ((((C:2.0,A:2.0):1.0,B:3.0):0.75,(D:2.5,E:2.5):1.25):0.6500000000000004,F:4.4)
Neighbor joining: (((C:2.0,(B:3.5,A:0.5):1.0):1.0,(D:2.5,E:1.5):1.0):2.5,F:2.5)
